In [1]:
import tensorflow as tf

In [1]:
%pip install tensorflow

     |████████████████████████████████| 394.3 MB 22 kB/s s eta 0:00:01     |████████▌                       | 104.7 MB 23.0 MB/s eta 0:00:13     |████████▉                       | 109.0 MB 23.0 MB/s eta 0:00:13     |█████████▌                      | 117.5 MB 30.8 MB/s eta 0:00:09     |██████████                      | 122.2 MB 30.8 MB/s eta 0:00:09     |██████████▉                     | 133.6 MB 30.8 MB/s eta 0:00:09     |███████████                     | 135.2 MB 30.8 MB/s eta 0:00:09�█████████▊                 | 180.8 MB 34.3 MB/s eta 0:00:07     |█████████████████▊              | 218.8 MB 30.9 MB/s eta 0:00:06     |████████████████████            | 245.6 MB 26.0 MB/s eta 0:00:06     |████████████████████▉           | 256.5 MB 26.0 MB/s eta 0:00:06     |███████████████████████         | 283.3 MB 28.5 MB/s eta 0:00:04     |███████████████████████▋        | 291.1 MB 32.7 MB/s eta 0:00:04�██████████▊        | 292.5 MB 32.7 MB/s eta 0:00:04
     |████████████████████████████████| 1.0 MB 

Note: you may need to restart the kernel to use updated packages.


In [3]:
a = tf.random.shuffle(tf.range(5))    #打乱顺序
a

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([2, 3, 4, 1, 0])>

In [5]:
tf.sort(a,direction='ASCENDING')     #升序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

In [6]:
tf.sort(a,direction='DESCENDING')     #降序排列

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 2, 1, 0])>

In [8]:
idx = tf.argsort(a)        #返回如何排序的
idx

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([4, 3, 0, 1, 2])>

In [9]:
tf.gather(a,idx)

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4])>

#### 高维数据排序

In [11]:
a = tf.random.uniform([3,3],maxval=10,dtype=tf.int32)
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 9],
       [9, 8, 7]])>

In [12]:
tf.sort(a)         #默认对最后一个维度进行排序

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[1, 2, 2],
       [3, 7, 9],
       [7, 8, 9]])>

In [13]:
tf.sort(a,direction='DESCENDING')

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [9, 7, 3],
       [9, 8, 7]])>

In [14]:
tf.sort(a,axis=0)     #对给定维度进行排序

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 7],
       [9, 8, 9]])>

#### Top_k 返回前几个最值及索引

In [15]:
a

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[2, 2, 1],
       [7, 3, 9],
       [9, 8, 7]])>

In [16]:
res = tf.math.top_k(a,2)    #返回两个最值

In [19]:
res.values     #返回最大的两个值

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[2, 2],
       [9, 7],
       [9, 8]])>

In [18]:
res.indices      #返回最大的两个值的索引

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[0, 1],
       [2, 0],
       [0, 1]])>

In [22]:
output = tf.random.normal([2,6])
output

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[-0.87764037,  0.8035606 , -1.1432153 ,  0.34809634,  0.89809996,
        -0.41597426],
       [-1.6270022 , -1.240233  ,  0.31657612, -0.5907291 ,  0.06888622,
        -0.8796589 ]], dtype=float32)>

In [27]:
output = tf.math.softmax(output)     #使六个数据概率总和为1
output

<tf.Tensor: shape=(2, 6), dtype=float32, numpy=
array([[0.14815383, 0.18879174, 0.14625038, 0.1693017 , 0.19445011,
        0.15305232],
       [0.14746293, 0.15101531, 0.19938007, 0.16162139, 0.18456003,
        0.15596028]], dtype=float32)>

In [28]:
tf.reduce_sum(output[0])

<tf.Tensor: shape=(), dtype=float32, numpy=1.0000001>

### Top_k Accuracy

In [5]:
def accuracy(output, target, topk=(1,)):
    maxk = max(topk)
    batch_size = target.shape[0]
    
    pred = tf.math.top_k(output,maxk).indices
    pred = tf.transpose(pred,perm=[1,0])        #转置
    target_ = tf.broadcast_to(target,pred.shape)
    correct = tf.equal(pred,target)
    
    res = []
    for k in topk:
        correct_k = tf.cast(tf.reshape(correct[:k],[-1]),dtype=tf.float32)       #先将矩阵打平，然后布尔转浮点
        correct_k = tf.reduce_sum(correct_k)           #计算所有预测对的次数
        acc = float(correct_k*100. / batch_size)
        res.append(acc)
    return res

In [6]:
output = tf.random.normal([10,6])
output = tf.math.softmax(output,axis=1)
target = tf.random.uniform([10],maxval=6,dtype=tf.int32)        #0-5的均匀分布作为目标值

print(output.numpy())
pred = tf.argmax(output,axis=1)     #得到10个预测值
print('pred',pred.numpy())
print('label',target.numpy())

acc = accuracy(output,target,topk=(1,2,3,4,5,6))
print(acc)
                    

[[0.130906   0.03784047 0.00361871 0.02464116 0.762372   0.04062167]
 [0.5932408  0.05387122 0.1025556  0.01377907 0.09901937 0.13753392]
 [0.10691354 0.32067314 0.21936873 0.02710809 0.2567868  0.06914965]
 [0.17471549 0.04824612 0.12160655 0.07682408 0.36541608 0.21319161]
 [0.06576606 0.25055325 0.38571578 0.12179352 0.02394885 0.15222242]
 [0.6735163  0.04113015 0.22935657 0.02399436 0.01426671 0.01773592]
 [0.07055986 0.03087542 0.07374463 0.2865004  0.22664729 0.31167242]
 [0.0245258  0.26187298 0.13328396 0.25841573 0.25986853 0.06203299]
 [0.27758157 0.18340456 0.38838583 0.04684191 0.07866074 0.02512534]
 [0.10291784 0.32314137 0.30706766 0.08131602 0.03029994 0.15525715]]
pred [4 0 1 4 2 0 5 1 2 1]
label [4 4 0 4 5 2 1 4 4 2]
[20.0, 50.0, 60.0, 90.0, 90.0, 100.0]
